In [1]:
import pandas as pd
import os

df = pd.read_csv(os.path.join('..', 'data', 'raw', 'train.csv'), index_col='id')
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')

df.head()

/tmp/ipykernel_342630/1755042591.py:4: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join('..', 'data', 'raw', 'train.csv'), index_col='id')


,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,p01,1900-01-01 06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
p01_1,p01,1900-01-01 06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
p01_2,p01,1900-01-01 06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
p01_3,p01,1900-01-01 06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
p01_4,p01,1900-01-01 07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, df['bg+1:00'], test_size=0.2, random_state=42)

# only use next line when training on the whole set is wanted
X_train = df

known_patients = X_train['p_num'].unique().tolist()

ts_lst = []
td_step = pd.Timedelta(minutes=5)

for n in range(72):
    column = f'bg-{(n*td_step).components.hours:01}:{(n*td_step).components.minutes:02}'
    ts = X_train[['time', column]].copy()
    ts['time'] -= n * td_step
    ts['time'] = ts['time'].dt.time
    ts = ts.rename(columns={column: 'bg'})
    ts_lst.append(ts)

ts = pd.concat(ts_lst)
TS = ts.groupby('time').agg({'bg': 'mean'})
TS = TS['bg']

In [3]:
#from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(df, df['bg+1:00'], test_size=0.2, random_state=42)

# only use next line when training on the whole set is wanted
#X_train = df

models = {}
for p_num in df['p_num'].unique():
    X_train_p = X_train[X_train['p_num'] == p_num]

    ts_lst = []
    td_step = pd.Timedelta(minutes=5)

    for n in range(72):
        column = f'bg-{(n*td_step).components.hours:01}:{(n*td_step).components.minutes:02}'
        ts = X_train_p[['time', column]].copy()
        ts['time'] -= n * td_step
        ts['time'] = ts['time'].dt.time
        ts = ts.rename(columns={column: 'bg'})
        ts_lst.append(ts)

    ts = pd.concat(ts_lst)
    TS_indi = ts.groupby('time').agg({'bg': 'mean'})
    TS_indi = TS_indi['bg']
    models[p_num] = TS_indi
models['generic'] = TS

In [4]:
from sklearn.metrics import root_mean_squared_error

def predict_simple(X):
    timecolumn = X['time'] + pd.Timedelta(hours=1)
    timecolumn = timecolumn.dt.time
    return timecolumn.apply(lambda x: TS[x])

def predict_simple_indi(X):
    temp = X[['time', 'p_num']].copy()
    temp['time'] = temp['time'] + pd.Timedelta(hours=1)
    temp['time'] = temp['time'].dt.time
    return temp.apply(lambda row: models[row['p_num']][row['time']] if row['p_num'] in known_patients
                                  else models['generic'][row['time']], axis=1)

def predict_offset(X):
    timecolumn = X['time']
    timecolumn = timecolumn.dt.time
    offset = X['bg-0:00'] - timecolumn.apply(lambda x: TS[x])
    offset = offset.fillna(0)
    timecolumn_future = X['time'] + pd.Timedelta(hours=1)
    timecolumn_future = timecolumn_future.dt.time
    return timecolumn_future.apply(lambda x: TS[x]) + offset

def predict_offset_indi(X):
    temp = X[['time', 'p_num']].copy()
    temp['time'] = temp['time'].dt.time
    offset = X['bg-0:00'] - temp.apply(lambda row: models[row['p_num']][row['time']] if row['p_num'] in known_patients
                                                   else models['generic'][row['time']], axis=1)
    offset = offset.fillna(0)
    temp['future'] = X['time'] + pd.Timedelta(hours=1)
    temp['future'] = temp['future'].dt.time
    return temp.apply(lambda row: models[row['p_num']][row['future']] if row['p_num'] in known_patients
                                  else models['generic'][row['time']], axis=1) + offset

def predict_factor(X):
    timecolumn = X['time']
    timecolumn = timecolumn.dt.time
    factor = X['bg-0:00']/timecolumn.apply(lambda x: TS[x])
    factor = factor.fillna(1)
    timecolumn_future = X['time'] + pd.Timedelta(hours=1)
    timecolumn_future = timecolumn_future.dt.time
    return factor * timecolumn_future.apply(lambda x: TS[x])

def predict_factor_indi(X):
    temp = X[['time', 'p_num']].copy()
    temp['time'] = temp['time'].dt.time
    factor = X['bg-0:00']/temp.apply(lambda row: models[row['p_num']][row['time']] if row['p_num'] in known_patients
                                                 else models['generic'][row['time']], axis=1)
    factor = factor.fillna(1)
    temp['future'] = X['time'] + pd.Timedelta(hours=1)
    temp['future'] = temp['future'].dt.time
    return factor * temp.apply(lambda row: models[row['p_num']][row['future']] if row['p_num'] in known_patients
                                           else models['generic'][row['time']], axis=1)

y_pred_simple = predict_simple(X_test)
y_pred_simple_indi = predict_simple_indi(X_test)
y_pred_offset = predict_offset(X_test)
y_pred_offset_indi = predict_offset_indi(X_test)
y_pred_factor = predict_factor(X_test)
y_pred_factor_indi = predict_factor_indi(X_test)

print('RMSE simple =', root_mean_squared_error(y_test, y_pred_simple))
print('RMSE simple indi =', root_mean_squared_error(y_test, y_pred_simple_indi))
print('RMSE offset =', root_mean_squared_error(y_test, y_pred_offset))
print('RMSE offset indi =', root_mean_squared_error(y_test, y_pred_offset_indi))
print('RMSE factor =', root_mean_squared_error(y_test, y_pred_factor))
print('RMSE factor indi =', root_mean_squared_error(y_test, y_pred_factor_indi))

RMSE simple = 2.973614099399331
RMSE simple indi = 2.725720450650036
RMSE offset = 2.3529408093021815
RMSE offset indi = 2.313047929284472
RMSE factor = 2.358339103673041
RMSE factor indi = 2.318621189470124


In [5]:
#import matplotlib.pyplot as plt
#%matplotlib inline

#plt.plot(TS.iloc[].index.astype('str'), TS)
#plt.ylabel('bg')
#plt.xlabel('time')
#plt.xticks([0, 72, 144, 216, 287])
#plt.title('Averaged blood glucose');

In [6]:
#df_test = pd.read_csv(os.path.join('..', 'data', 'raw', 'test.csv'), index_col='id')
#df_test['time'] = pd.to_datetime(df_test['time'], format='%H:%M:%S')

#df_test.head()

In [7]:
df_test = pd.read_csv(os.path.join('..', 'data', 'raw', 'test.csv'), index_col='id')
df_test['time'] = pd.to_datetime(df_test['time'], format='%H:%M:%S')

df_test['bg+1:00'] = predict_simple(df_test)
df_test['bg+1:00'].to_csv(os.path.join('..', 'data', 'raw', 'av.csv'))
df_test['bg+1:00'] = predict_simple_indi(df_test)
df_test['bg+1:00'].to_csv(os.path.join('..', 'data', 'raw', 'av_indi.csv'))

df_test['bg+1:00'] = predict_offset(df_test)
df_test['bg+1:00'].to_csv(os.path.join('..', 'data', 'raw', 'av_off.csv'))
df_test['bg+1:00'] = predict_offset_indi(df_test)
df_test['bg+1:00'].to_csv(os.path.join('..', 'data', 'raw', 'av_off_indi.csv'))

df_test['bg+1:00'] = predict_factor(df_test)
df_test['bg+1:00'].to_csv(os.path.join('..', 'data', 'raw', 'av_factor.csv'))
df_test['bg+1:00'] = predict_factor_indi(df_test)
df_test['bg+1:00'].to_csv(os.path.join('..', 'data', 'raw', 'av_factor_indi.csv'))